In [2]:

import pandas as pd
import numpy as np
import math
import itertools

def entrop(data):
    uniq = np.unique(data)
    
    den = len(data)
    entr = 0
    
    for i in range(len(uniq)):
        num = np.count_nonzero(data==uniq[i])
        if num != 0:
            entr = entr - (num)/den*math.log2(num/den)
    
    return entr
    
def calcInfoG(x,y):
    gainv = ent_whole
    val = np.unique(x)

    for i in range(len(val)):
        gainv -= np.count_nonzero(x == val[i])*entrop(y[x == val[i]])/len(x)
        
    return gainv


def calcspl(x,y):
    spl = 0
    val = np.unique(x)

    for i in range(len(val)):
        spl -= np.count_nonzero(x==val[i])/len(x)*math.log2(np.count_nonzero(x==val[i])/len(x))
        
    return spl

def calcgin(x):
    ans = 1
    uniqv = np.unique(x)
    
    for i in range(len(uniqv)):
        ans -= math.pow(np.count_nonzero(x == uniqv[i])/len(x),2)
        
    return ans

def calcginatt(x,y):
    
    giniv = 0;
    uniq = np.unique(x)
    for i in range(len(uniq)):
        giniv += np.count_nonzero(x == uniq[i])/len(x) * calcgin(y[x == uniq[i]])
        
    return giniv

def calcgi(x,y):
    
    uniq = np.unique(x)
    
    val_fre = len(uniq)
    ginv = 1000
    arr2 = []
    
    if val_fre == 2:
        return calcginatt(x,y), uniq
        
    for i in range(2,val_fre):
        arr = itertools.combinations(uniq,i)
        
        for subs in arr:
            s1 = np.array(subs)
            
            data_1 = np.array([y[ind] for ind in range(len(y)) if x[ind] in s1])
            data_2 = np.array([y[ind] for ind in range(len(y)) if x[ind] not in s1])
            
            gin1 = calcgin(data_1)
            gin2 = calcgin(data_2)
           
            gv = len(data_1)/len(x)*gin1 + len(data_2)/len(x)*gin2
          
            if gv<ginv:
                arr2 = s1
                ginv =gv
      
    return ginv,np.array(arr2)
    
dataset = pd.read_csv('weatherpred.csv')
X = np.array(dataset)

param = X[:,0:-1]
classv = X[:,-1]


ent_whole = entrop(classv)

print('Entropy: ', ent_whole)
gain = [calcInfoG(param[:,j],classv) for j in range(len(param[0]))]
print('Information gain: ', gain)

rootind = np.argmax(gain)
print('Root node using information gain: ', dataset.columns[rootind])

splinf = [calcspl(param[:,j],classv) for j in range(len(param[0]))]

gainrat = np.array(gain)/np.array(splinf)

print('Gain ratio: ', gainrat)
rootind2 = np.argmax(gainrat)
print('Root node using gain ratio: ', dataset.columns[rootind2])

ginient = calcgin(classv)
gnind = [calcginatt(param[:,j],classv) for j in range(len(param[0]))]

gaingin = ginient - np.array(gnind)
print('Gain using gini index: ', gaingin)
rootind3 = np.argmax(gaingin)
print('Root node using gini index: ', dataset.columns[rootind3])

print('Gini index using combination: ')
print(np.array([calcgi(param[:,j],classv) for j in range(len(param[0]))]))


Entropy:  0.9402859586706309
Information gain:  [0.24674981977443905, 0.06423863677845904, 0.15183550136234142, 0.04812703040826927]
Root node using information gain:  Feat1
Gain ratio:  [0.15642756 0.0359186  0.1518355  0.04884862]
Root node using gain ratio:  Feat1
Gain using gini index:  [0.11632653 0.03061224 0.09183673 0.03061224]
Root node using gini index:  Feat1
Gini index using combination: 
[[0.35714285714285715 array(['Rain', 'Sunny'], dtype='<U5')]
 [0.43956043956043955 array(['Cool', 'Hot', 'Mild'], dtype='<U4')]
 [0.3673469387755103 array(['High', 'Normal'], dtype=object)]
 [0.42857142857142855 array(['Strong', 'Weak'], dtype=object)]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
